In [12]:
import findspark
findspark.init()

In [13]:
from pyspark.sql import SparkSession

In [32]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [31]:
USER_NAME = "sourcedb1"
PASS_WORD = "sourcedb1"
HOST_NAME = "postgres-source"
DB_NAME = "sourcedb"

CONN_STRING = f"postgresql+psycopg2://{USER_NAME}:{PASS_WORD}@{HOST_NAME}/{DB_NAME}"
engine = create_engine(CONN_STRING)

In [16]:
appName = "pyspark_postgres"
master = "local"

In [17]:
spark = SparkSession.builder \
        .master(master) \
        .appName(appName) \
        .getOrCreate()

21/09/07 09:49:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Customers Table

In [21]:
customers_df = pd.read_sql('SELECT * FROM ecommerce.customers', con=engine)

In [23]:
customers_sdf = spark.createDataFrame(customers_df)

In [25]:
print(customers_sdf.schema)

StructType(List(StructField(customer_id,StringType,true),StructField(customer_unique_id,StringType,true),StructField(customer_zip_code_prefix,StringType,true),StructField(customer_city,StringType,true),StructField(customer_state,StringType,true)))


## Orders Table

In [26]:
orders_df = pd.read_sql('SELECT * FROM ecommerce.orders', con=engine)

In [27]:
orders_sdf = spark.createDataFrame(orders_df)

In [30]:
pprint(orders_sdf.schema)

StructType(List(StructField(order_id,StringType,true),StructField(customer_id,StringType,true),StructField(order_status,StringType,true),StructField(order_purchase_timestamp,TimestampType,true),StructField(order_approved_at,TimestampType,true),StructField(order_delivered_carrier_date,TimestampType,true),StructField(order_delivered_customer_date,TimestampType,true),StructField(order_estimated_delivery_date,TimestampType,true)))


## Output as Parquet

In [39]:
customers_sdf.write.parquet('/filesystem/customers.parquet', compression='snappy')

21/09/07 10:15:42 WARN TaskSetManager: Stage 4 contains a task of very large size (9655 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
orders_sdf.write.parquet('/filesystem/orders.parquet', compression='snappy')